In [94]:
import random
import numpy as np
import pandas as pd

ARRIVAL = 0.4  # higher -> more arrivals
SERVICE = 1  # higher -> less service time

SIM_TIME = 20

TYPE1 = 1
TYPE2 = 2

N_SERVERS = 2

random.seed(42)

In [95]:
u_Id = 0
users = 0

# Future Event Set
FES = []
queue = []

# simulation time
time = 0

# event counter
event = 0
arrivals = 0
departures = 0

In [96]:
columns = ['event', 's_Id', 'time', 'type',
           'queue', 'arr cust', 'depar cust', 'in line']
columns_s = ['time', 'type', 'queue', 'in line']

In [97]:
# Client

class Client:
    def __init__(self, u_Id, type, arrival_time, s_Id) -> None:
        self.u_Id = u_Id
        self.type = type
        self.arrival_time = arrival_time
        self.s_Id = s_Id

    def getType(self):
        return self.type

    def getArrivalTime(self):
        return self.arrival_time

    def getId(self):
        return self.u_Id

    def getS_Id(self):
        return self.s_Id

In [98]:
# find the server with the lowest number of clients in queue

def selectServer():
    global servers
    minId = -1
    nClient = float('inf')
    for s in servers:
        if s.getNClients() < nClient:
            nClient = s.getNClients()
            minId = s.getId()

    return minId


In [99]:
# Server

class Server:
    def __init__(self, s_Id, FES, columns_s) -> None:
        self.s_Id = s_Id
        self.FES = FES
        self.nClients = 0
        self.df = pd.DataFrame(columns=columns_s)
        self.columns = columns_s

    def addClient(self):
        self.nClients += 1

    def removeClient(self):
        self.nClients -= 1

    def getNClients(self):
        return self.nClients

    def getId(self):
        return self.s_Id

    def getDf(self):
        return self.df

    def arrival(self, time, FES, queue):
        global users
        global servers
        global u_Id
        print("ARRIVAL at time: \t{}".format(time))
        # sample time until next event
        inter_arrival = random.expovariate(1.0/ARRIVAL)
        #schedule next arrival
        FES.append((time + inter_arrival, "arrival", self.s_Id))
        # select the less busy server
        s_Id = selectServer()
        # create a record for the client
        client = Client(u_Id, TYPE1, time, s_Id)
        # assign the client to the less busy server
        servers[s_Id].addClient()
        # update state variables
        users += 1
        # change id of next user
        u_Id += 1
        print("S [{}]: new CLIENT created of type [{}] at time [{}]".format(s_Id, client.getType(), client.getArrivalTime()))
        # insert record in the queue
        queue.append(client)

        # update dataframe
        new_df = pd.DataFrame([
            [float(time), "arrival", len(queue), self.nClients]
        ], columns=self.columns)
        self.df = pd.concat([self.df, new_df])

        # if server is idle start the service
        if servers[s_Id].getNClients() == 1:
            # sample the service time
            service_time = random.expovariate(1.0/SERVICE)

            # print("S [{}]: new DEPARTURE scheduled at time: {}".format(self.s_Id, time + service_time))

            # schedule when the client will finish the server
            FES.append((time + service_time, "departure", s_Id))

    def departure(self, time, FES, queue):
        global users
        # get the first element from the queue
        client = queue.pop(0)
        print("S [{}]: CLIENT {} of type [{}] created at time [{}] departed at time [{}]".format(self.s_Id, client.getId(), client.getType(), client.getArrivalTime(), time))
        # remove client from server
        self.removeClient()
        # update state variable
        users -= 1

        # update dataframe
        new_df = pd.DataFrame([
            [float(time), "departure", len(queue), self.nClients]
        ], columns=self.columns)
        self.df = pd.concat([self.df, new_df])

        # see wheather there are more clients to serve in the line
        if self.getNClients() > 0:
            # sample service time
            service_time = random.expovariate(1.0/SERVICE)
            # schedule when the client will finish the server
            FES.append((time + service_time, "departure", self.s_Id))

In [100]:
# schedule the first arrival at t=0 to server 0
FES.append((time, "arrival", 0))

#create dataframe to store informations
df = pd.DataFrame(columns=columns)

# create servers
servers = []
for s_Id in range(0, N_SERVERS):
    servers.append(Server(s_Id, FES, columns_s))

while time < SIM_TIME:
    # sort FES in order to have events in cronological order
    FES.sort(key=lambda x: x[0])
    (time, event_type, s_Id) = FES.pop(0)

    print("users in line: \t{}".format(users))

    if event_type == "arrival":
        event += 1
        arrivals += 1
        new_df = pd.DataFrame([
            [event, s_Id, float(time), "arrival", len(queue), arrivals, departures, users]
        ], columns=columns)

        servers[s_Id].arrival(time, FES, queue)

    elif event_type == "departure":
        event += 1
        departures += 1
        new_df = pd.DataFrame([
            [event, s_Id, float(time), "departure", len(queue), arrivals, departures, users]
        ], columns=columns)

        servers[s_Id].departure(time, FES, queue)

    df = pd.concat([df, new_df])
    print()


users in line: 	0
ARRIVAL at time: 	0
S [0]: new CLIENT created of type [1] at time [0]

users in line: 	1
S [0]: CLIENT 0 of type [1] created at time [0] departed at time [0.02532883904273889]

users in line: 	0
ARRIVAL at time: 	0.4080241149099204
S [0]: new CLIENT created of type [1] at time [0.4080241149099204]

users in line: 	1
ARRIVAL at time: 	0.5366737405399067
S [1]: new CLIENT created of type [1] at time [0.5366737405399067]

users in line: 	2
S [0]: CLIENT 1 of type [1] created at time [0.4080241149099204] departed at time [0.6606103005800339]

users in line: 	1
ARRIVAL at time: 	1.0701108096631398
S [0]: new CLIENT created of type [1] at time [1.0701108096631398]

users in line: 	2
S [0]: CLIENT 2 of type [1] created at time [0.5366737405399067] departed at time [1.161063214269706]

users in line: 	1
S [1]: CLIENT 3 of type [1] created at time [1.0701108096631398] departed at time [1.6658467493384994]

users in line: 	0
ARRIVAL at time: 	1.961026049483692
S [0]: new CLIENT

In [101]:
df

,event,s_Id,time,type,queue,arr cust,depar cust,in line
0,1,0,0.000000,arrival,0,1,0,0
0,2,0,0.025329,departure,1,1,1,1
0,3,0,0.408024,arrival,0,2,1,0
0,4,0,0.536674,arrival,1,3,1,1
0,5,0,0.660610,departure,2,3,2,2
...,...,...,...,...,...,...,...,...
0,92,0,19.398176,departure,13,52,40,13
0,93,0,19.887684,arrival,12,53,40,12
0,94,0,19.913938,arrival,13,54,40,13
0,95,0,19.946635,departure,14,54,41,14


In [102]:
servers[0].getDf()

,time,type,queue,in line
0,0.000000,arrival,1,1
0,0.025329,departure,0,0
0,0.408024,arrival,1,1
0,0.536674,arrival,2,1
0,0.660610,departure,1,0
...,...,...,...,...
0,19.398176,departure,12,6
0,19.887684,arrival,13,7
0,19.913938,arrival,14,7
0,19.946635,departure,13,6


In [103]:
servers[1].getDf()

,time,type,queue,in line
0,1.665847,departure,0,0
0,2.500623,departure,2,0
0,2.958512,departure,1,0
0,3.030937,departure,1,0
0,4.885303,departure,1,0
0,6.734649,departure,1,0
0,10.835728,departure,10,4
0,13.594881,departure,12,6
0,14.748839,departure,11,5
0,15.008969,departure,13,6
